<a href="https://colab.research.google.com/github/forbeso/JaCovid/blob/develop/Jamaica_Covid_19_Cases_By_Parish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import folium
from folium import FeatureGroup, LayerControl, Map, Marker, Choropleth, ColorMap
import json
import os
import pathlib as pl
from bs4 import BeautifulSoup
import urllib.request,json
import requests
from folium.plugins import MarkerCluster
import re
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#get data from api 

url ='https://admin.jamcovid19.moh.gov.jm/public/api/statistics?type=1'

r = requests.get(url)
dictr = r.json()
recs = dictr['data']['parishes_wise_report']
df = pd.json_normalize(recs)
#print(df)

frame = pd.DataFrame(data=df)

#select columns needed
ParishData = frame.loc[:,['parish','total_tests_positive','number_of_deaths','number_of_people_recovered']]

#rename columns
ParishData.rename(columns={'parish':'Parish',
                           'total_tests_positive':'Confirmed','number_of_deaths':'Deaths', 
                           'number_of_people_recovered':'Recoveries'}, inplace=True)
ParishData


In [0]:
ParishData_new = ParishData
ParishData_new.at[0,'Parish'] = 'Hanover'
ParishData_new.at[1,'Parish'] = 'St. Elizabeth'
ParishData_new.at[2,'Parish'] = 'St. James'
ParishData_new.at[7,'Parish'] = 'St. Ann'
ParishData_new.at[8,'Parish'] = 'St. Catherine'
ParishData_new.at[9,'Parish'] = 'St. Mary'
ParishData_new.at[11,'Parish'] = 'St. Andrew'
ParishData_new.at[12,'Parish'] = 'St. Thomas'
# ParishData_new = pd.DataFrame({'Parish':['Hanover','St. Elizabeth','St. James','Trelawny','Westmoreland','Clarendon','Manchester','St. Ann','St. Catherine','St. Mary','Portland','St. Andrew','St. Thomas']})

ParishData_new

In [0]:
#use folium to get Jamaica's Map
jamaica = folium.Map(
    location=[18.1096,-77.2975],
    zoom_start=9,
    tiles='Mapbox Bright'
)

In [0]:
ParishData_new.dtypes

In [0]:
#change data types
ParishData_new['Parish'] = ParishData_new['Parish'].astype(str)

ParishData_new['Confirmed'] = pd.to_numeric(ParishData_new['Confirmed'])
ParishData_new['Recoveries'] = pd.to_numeric(ParishData_new['Recoveries'])
ParishData_new['Deaths'] = pd.to_numeric(ParishData_new['Deaths'])

ParishData_new.dtypes

In [0]:
#get geojosn file with Jamaica's Parishes/Boundaries
geojson = os.path.join(pl.Path.cwd(),'data', '/content/drive/My Drive/Colab Notebooks/Data/JMParishes.json')
jamaica_geo = json.load(open(geojson))

In [0]:
for j in range(13):
  for i in range(len(ParishData_new['Parish'])):
    if ((jamaica_geo['features'][j]['properties']['PARISH']) == (ParishData_new['Parish'][i])):
      jamaica_geo['features'][j]['properties']['Confirmed'] = ParishData_new['Confirmed'][i]
      jamaica_geo['features'][j]['properties']['Recoveries'] = ParishData_new['Recoveries'][i]
      jamaica_geo['features'][j]['properties']['Deaths'] = ParishData_new['Deaths'][i]
  print (jamaica_geo['features'][j]['properties'])

  

In [0]:
#CONVERT FROM NUMPY INT64 TO REGULAR INT
for x in range(13):
  jamaica_geo['features'][x]['properties']['Confirmed'] = np.int(jamaica_geo['features'][x]['properties']['Confirmed'])
  jamaica_geo['features'][x]['properties']['Recoveries'] = np.int(jamaica_geo['features'][x]['properties']['Recoveries'])
  jamaica_geo['features'][x]['properties']['Deaths'] = np.int(jamaica_geo['features'][x]['properties']['Deaths'])


In [0]:
print(type(jamaica_geo['features'][0]['properties']['Deaths']))

In [0]:
#create Choropleth Map of Jamaica 
choro=Choropleth(
    geo_data = jamaica_geo,
    data = ParishData_new,
    name = 'choro',
    columns = ['Parish','Confirmed'],
    key_on='feature.properties.PARISH',
    fill_color='YlOrRd',
    fill_opacity=0.9,
    legend_name = 'Confirmed Corona Virus Cases by Parish',
    reset=True
).add_to(jamaica)

In [0]:
choro.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['PARISH', 'Confirmed','Deaths','Recoveries'], labels=True))

In [0]:
epi = ParishData_new.iloc[8]['Confirmed']
info = str(epi) +' Cases'
print (info)

folium.CircleMarker(
    location=[18.1096,-77],
    radius=20,
    fill=True,
).add_child(folium.Popup(info)).add_to(jamaica)

# folium.Marker(
#     location=[18.1096,-77],
#     # radius=10,
#     # fill=True,
# ).add_child(folium.Popup(info)).add_to(jamaica)

In [0]:
ParishData_new.dtypes

In [0]:
jamaica

In [0]:
jamaica.save('jamaica.html')